In [1]:
import pvporcupine
import sounddevice as sd

# Configuración de Porcupine con access_key
porcupine = pvporcupine.create(
    access_key="qEcRoI9pyPji4cEIXkMNz4+G34KlNQMO5mouLLJzGsvb5PF5fuaKlg==",  # Reemplaza con tu clave de acceso
    keyword_paths=["resources/wakeword.ppn"],
    model_path="resources/porcupine_params_es.pv" 
)

def wake_word_detected():
    print("Wake word detectado. Escuchando tu comando...")
import numpy as np

def audio_callback(indata, frames, time, status):
    if len(indata) != porcupine.frame_length:
        print(f"Ajustando frame length. Original: {len(indata)}, esperado: {porcupine.frame_length}")
        # Interpolamos para ajustar el tamaño del frame
        factor = porcupine.frame_length / len(indata)
        indata = np.interp(
            np.arange(0, len(indata) * factor) / factor,  # Nuevas posiciones
            np.arange(len(indata)),  # Posiciones originales
            indata[:, 0]  # Solo el primer canal
        ).astype("int16")

    pcm = indata  # Ya está ajustado
    keyword_index = porcupine.process(pcm)
    if keyword_index >= 0:
        wake_word_detected()



In [2]:
import whisper
import sounddevice as sd
import numpy as np

# Cargar el modelo de Whisper
model = whisper.load_model("base")

def transcribe_audio():
    print("Escuchando comando...")
    duration = 5  # Duración en segundos
    audio = sd.rec(int(duration * 16000), samplerate=16000, channels=1, dtype='float32')
    sd.wait()  # Esperar a que termine la grabación
    
    # Guardar el audio en un archivo temporal
    np.save("audio_temp.npy", audio)
    print("Transcribiendo...")
    
    # Usar Whisper para convertir audio a texto
    result = model.transcribe("audio_temp.npy", language="es")
    return result['text']


c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

In [3]:
def process_command(command):
    print(f"Comando recibido: {command}")
    genre = None
    runtime = None

    # Ejemplo de procesamiento básico
    if "amor" in command:
        genre = "romance"
    if "menos de 1:30" in command or "corta" in command:
        runtime = 90

    return {
        "genre": genre,
        "runtime": runtime
    }


In [4]:
import requests

# URL del servidor Flask
BASE_URL = "http://127.0.0.1:5000"

# Obtener la lista de seguimiento (después de autenticarse)
response = requests.get(f"{BASE_URL}/get_watchlist")
print(response.json())


{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/8mjYwWT50GkRrrRdyHzJorfEfcl.jpg', 'genre_ids': [28, 12, 18], 'id': 558449, 'original_language': 'en', 'original_title': 'Gladiator II', 'overview': 'Years after witnessing the death of the revered hero Maximus at the hands of his uncle, Lucius is forced to enter the Colosseum after his home is conquered by the tyrannical Emperors who now lead Rome with an iron fist. With rage in his heart and the future of the Empire at stake, Lucius must look to his past to find strength and honor to return the glory of Rome to its people.', 'popularity': 2557.256, 'poster_path': '/2cxhvwyEwRlysAmRH4iodkvo0z5.jpg', 'release_date': '2024-11-13', 'title': 'Gladiator II', 'video': False, 'vote_average': 6.8, 'vote_count': 329}, {'adult': False, 'backdrop_path': '/18TSJF1WLA4CkymvVUcKDBwUJ9F.jpg', 'genre_ids': [27, 53, 9648], 'id': 1034541, 'original_language': 'en', 'original_title': 'Terrifier 3', 'overview': "Five years after surviving Art the

In [5]:
def speak_recommendations(movies):
    engine.say("Estas son las películas que encontré para ti:")
    for movie in movies[:3]:  # Limita a 3 resultados
        title = movie["title"]
        overview = movie.get("overview", "Sin sinopsis disponible.")
        rating = movie.get("rating", "Sin calificación.")
        engine.say(f"{title}: {overview}. Calificación promedio: {rating}.")
    engine.runAndWait()


In [6]:
def detect_wake_word():
    def audio_callback(indata, frames, time, status):
        if status:
            print(f"Estado del stream: {status}")
        
        # Verifica si el tamaño del frame es correcto
        if len(indata) == porcupine.frame_length:
            pcm = indata[:, 0]  # Procesa solo el canal mono
            keyword_index = porcupine.process(pcm)
            if keyword_index >= 0:
                wake_word_detected()
        else:
            print(f"Frame length inválido: esperado {porcupine.frame_length}, recibido {len(indata)}")

    # Configurar el flujo de audio con blocksize
    with sd.InputStream(
        callback=audio_callback,
        channels=1,  # Mono
        samplerate=porcupine.sample_rate,  # 16000 Hz
        blocksize=porcupine.frame_length,  # Asegurar el tamaño del bloque
        dtype="int16",  # Formato esperado por Porcupine
    ):
        print("Escuchando activación...")
        while True:
            pass



In [7]:
import pvporcupine
import sounddevice as sd
import numpy as np
import time
from scipy.io.wavfile import write
import os

DEVICE_INDEX = 5  # Ajustar al índice correcto según tu micrófono
running = True  # Bandera para controlar el bucle principal

# Configuración de Porcupine
try:
    porcupine = pvporcupine.create(
        access_key="qEcRoI9pyPji4cEIXkMNz4+G34KlNQMO5mouLLJzGsvb5PF5fuaKlg==",
        keyword_paths=["resources/wakeword.ppn"],
        model_path="resources/porcupine_params_es.pv"
    )
except Exception as e:
    print(f"Error al inicializar Porcupine: {e}")
    exit(1)

def wake_word_detected():
    global running
    print("Wake word detectado. Preparando para escuchar el comando...")
    grabar_comando()
    running = False  # Detener el bucle principal

def grabar_comando():
    print("Grabando audio por 5 segundos...")
    try:
        duration = 5
        audio = sd.rec(int(duration * 16000), samplerate=16000, channels=1, dtype='int16', device=DEVICE_INDEX)
        sd.wait()
        print("Grabación completada.")
        
        # Guardar el archivo en formato estándar para Whisper
        temp_audio_path = "temp_audio.wav"
        write(temp_audio_path, 16000, audio)
        print(f"Audio guardado en {os.path.abspath(temp_audio_path)}")
    except Exception as e:
        print(f"Error al grabar audio: {e}")

def audio_callback(indata, frames, time, status):
    if status:
        print(f"Estado del stream: {status}")
    try:
        pcm = indata[:, 0]
        keyword_index = porcupine.process(pcm)
        if keyword_index >= 0:
            wake_word_detected()
    except Exception as e:
        print(f"Error en el callback: {e}")

def detect_wake_word():
    global running
    print("Iniciando el sistema de detección de wake word...")
    try:
        with sd.InputStream(
            device=DEVICE_INDEX,
            callback=audio_callback,
            channels=1,
            samplerate=porcupine.sample_rate,
            blocksize=porcupine.frame_length,
            dtype="int16",
            latency="low"
        ) as stream:
            print("Escuchando activación...")
            while running:
                time.sleep(0.1)
    except KeyboardInterrupt:
        print("\nInterrupción detectada. Finalizando el programa...")
    except Exception as e:
        print(f"Error en el sistema de detección: {e}")
    finally:
        porcupine.delete()
        print("Sistema de detección finalizado.")

if __name__ == "__main__":
    detect_wake_word()


Iniciando el sistema de detección de wake word...
Escuchando activación...
Wake word detectado. Preparando para escuchar el comando...
Grabando audio por 5 segundos...
Grabación completada.
Audio guardado en c:\Users\alexc\OneDrive - Universidad Politécnica de Madrid\Escritorio\CDIA\Proyectos\Film_recomendations\temp_audio.wav
Sistema de detección finalizado.


In [8]:
import whisper
import os

os.environ["PATH"] += os.pathsep + r"C:\ffmpeg\bin"
# Cargar el modelo de Whisper
print("Cargando modelo de Whisper...")
model = whisper.load_model("base")

def transcribe_audio(filename):
    """Transcribir audio usando Whisper"""
    if os.path.exists(filename):
        print(f"Archivo encontrado: {os.path.abspath(filename)}")
        print("Procesando transcripción...")
        try:
            # Asegúrate de que el archivo está accesible y tiene el formato correcto
            result = model.transcribe(filename, language="es")
            print(f"Transcripción: {result['text']}")
        except Exception as e:
            print(f"Error al transcribir el audio: {e}")
    else:
        print(f"Error: El archivo {filename} no existe o no es accesible.")

# Ruta del archivo generado en la parte anterior
wav_filename = "temp_audio.wav"
transcribe_audio(wav_filename)


Cargando modelo de Whisper...


c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

Archivo encontrado: c:\Users\alexc\OneDrive - Universidad Politécnica de Madrid\Escritorio\CDIA\Proyectos\Film_recomendations\temp_audio.wav
Procesando transcripción...


c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción:  Quiero ver una peli de antes del año 2004.
